In [1]:
import folium
from folium.plugins import HeatMap
import numpy as np
import pandas as pd
import os
current_directory = os.getcwd()

In [2]:
location=[32.7003, -90.6783]
weather_data = pd.read_csv('/Users/abdullaahrobins/Documents/GitHub/CS506Spring2021Repository/NaturalDisasterProject/Datasets/Tornado_Data_2010-2020.csv')
# folium.Map(location=location, zoom_start=3)
# heat_df = weather_data[['BEGIN_LAT', 'BEGIN_LON']]
# heat_df = heat_df.dropna(axis=0, subset=['BEGIN_LAT','BEGIN_LON'])
# HeatMap(data=weather_data.groupby(['BEGIN_LAT', 'BEGIN_LON']).mean().reset_index().values.tolist(), radius=8, max_zoom=4).add_to(base_map)
# base_map.save('tornado_heat_map.html')

In [ ]:
# info = weather_data.info()
# state_occ = weather_data['STATE_ABBR'].value_counts()
# df = weather_data['STATE_ABBR'].value_counts().rename_axis('states').reset_index(name='counts')

In [ ]:
# url = (
#     "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
# )
# state_geo = f"{url}/us-states.json"

# m = folium.Map(location=location, zoom_start=3)

# folium.Choropleth(
#     geo_data=state_geo,
#     name="choropleth",
#     data=df,
#     columns=["states", "counts"],
#     key_on="feature.id",
#     fill_color="YlGn",
#     fill_opacity=0.7,
#     line_opacity=0.2,
#     legend_name="Tornado Occurrances",
# ).add_to(m)

# folium.LayerControl().add_to(m)

In [ ]:
# heat_data = [[row['BEGIN_LAT'],row['BEGIN_LON']] for index, row in heat_df.iterrows()]
# HeatMap(heat_data).add_to(base_map)

In [ ]:
# base_map.save('heatmap_weather.html')
# base_map.save('choropleth_weather.html')

In [3]:
weather_data['BEGIN_DATE'] = pd.to_datetime(weather_data['BEGIN_DATE'])

In [5]:
import requests
from datetime import timedelta
import json
'''
Formatting:
'start': '2010-01-01',
'end': '2018-12-31',
'''
c = 0
frames = []
for index, row in weather_data.iterrows():
    c+=1
    if c>50:
        break
    end_date = row.BEGIN_DATE
    begin_date = end_date - timedelta(weeks=8)
    lat = row.BEGIN_LAT
    lon = row.BEGIN_LON
    uid = row.EVENT_ID
    r = requests.get('https://api.oikolab.com/weather',
                     params={'param': ['temperature', 'wind_speed', 'surface_solar_radiation', 'relative_humidity', 'surface_pressure', 'total_precipitation'],
                             'start': begin_date,
                             'end': end_date,
                             'freq': 'D',
                             'lat': lat,
                             'lon': lon,
                             'api-key': 'dca39edd13e4410180be9e852e37fbab'}
                     )
    w_data = json.loads(r.json()['data'])
    history_df = pd.DataFrame(index=pd.to_datetime(w_data['index'], 
                                       unit='s'),
                  data=w_data['data'],
                  columns=w_data['columns'])
    history_df['latitude'] = lat
    history_df['longitude'] = lon
    history_df['event_id'] = uid
    frames.append(history_df)
result = pd.concat(frames)
result.index.name = 'datetime'
result.to_csv("historical_data_firsthalf.csv", encoding='utf-8', index=False)

In [9]:
import requests
from datetime import timedelta
import json
from itertools import islice

'''
Formatting:
'start': '2010-01-01',
'end': '2018-12-31',
'''
c = 0
frames = []
for index, row in islice(weather_data.iterrows(),50,None):
    end_date = row.BEGIN_DATE
    begin_date = end_date - timedelta(weeks=8)
    lat = row.BEGIN_LAT
    lon = row.BEGIN_LON
    uid = row.EVENT_ID
    r = requests.get('https://api.oikolab.com/weather',
                     params={'param': ['temperature', 'wind_speed', 'surface_solar_radiation', 'relative_humidity', 'surface_pressure', 'total_precipitation'],
                             'start': begin_date,
                             'end': end_date,
                             'freq': 'D',
                             'lat': lat,
                             'lon': lon,
                             'api-key': 'cbc6422e53084e909bf9934fcd6d8adc'}
                     )
    w_data = json.loads(r.json()['data'])
    history_df = pd.DataFrame(index=pd.to_datetime(w_data['index'], 
                                       unit='s'),
                  data=w_data['data'],
                  columns=w_data['columns'])
    history_df['latitude'] = lat
    history_df['longitude'] = lon
    history_df['event_id'] = uid
    frames.append(history_df)
result2 = pd.concat(frames)
result2.index.name = 'datetime'
result2.to_csv("historical_data_second.csv", encoding='utf-8')

In [10]:
result_fin = pd.concat([result,result2])
result_fin.to_csv("historical_data_final.csv", encoding='utf-8')


In [11]:
result_fin

,temperature (degC),wind_speed (m/s),surface_solar_radiation (W/m^2),relative_humidity (0-1),surface_pressure (Pa),total_precipitation (mm of water equivalent),latitude,longitude,event_id
datetime,,,,,,,,,
2010-02-27,8.50,1.51,233,0.50,101170,0.00,32.7003,-90.6783,224551
2010-02-28,8.04,2.59,75,0.62,100953,0.19,32.7003,-90.6783,224551
2010-03-01,5.64,4.81,89,0.76,100742,0.30,32.7003,-90.6783,224551
2010-03-02,5.33,3.02,182,0.59,101282,0.00,32.7003,-90.6783,224551
2010-03-03,5.36,0.77,242,0.56,101601,0.00,32.7003,-90.6783,224551
...,...,...,...,...,...,...,...,...,...
2020-07-04,26.76,3.30,331,0.61,97023,0.00,46.1100,-95.9000,898307
2020-07-05,24.55,1.61,233,0.69,97004,0.11,46.1100,-95.9000,898307
2020-07-06,24.38,1.65,287,0.68,96966,0.01,46.1100,-95.9000,898307


In [8]:
for index, row in islice(weather_data.iterrows(),50,None):
    print(row.EVENT_ID)

300457
315334
300461
296617
315837
315858
315875
315859
313004
353589
359691
359697
359896
359912
359916
359915
369474
366383
433413
443090
457695
451506
451537
451572
472670
476100
483775
482450
504760
508158
514013
514014
514042
514022
514044
524717
574337
610234
610235
610236
610237
610239
611942
606471
633058
626912
677516
683249
813640
816403
830798
883016
887908
902831
902837
902843
902844
889068
899721
898306
898307
